## De novo sequence assembly from short reads using index-assisted greedy shortest common superstring algorithm



### 1. Sequencing reads download

In [4]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

--2016-07-20 17:16:04--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq
Resolving d28rh4a8wq0iu5.cloudfront.net... 52.84.247.114, 52.84.247.186, 52.84.247.38, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|52.84.247.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395781 (387K) [video/m2ts]
Saving to: 'ads1_week4_reads.fq'

ads1_week4_reads.fq 100%[=====================>] 386.50K   646KB/s   in 0.6s   

2016-07-20 17:16:06 (646 KB/s) - 'ads1_week4_reads.fq' saved [395781/395781]



### 2. Python function definitions for reading sequece read, indexing, and assembly

In [1]:
## use k-mer index on reads to assist genome assembly
## improved performance on brute-force scs assembly

def readFastq(filename):
    reads = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            fh.readline() # skip quality line
            if len(seq) == 0:
                break
            reads.append(seq)
    return reads


def overlap(a, b, min_length=1):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def allOverlapPairs(reads, k=30, min_overlap=1):    
    ## returns the pairs in list with the highest overlength as the 1st element
    kIndex = {}
    pairs = []
    
    for read in reads:
    # generate the k-mer index in the format of k-mer:a set of sequences
        for i in range(len(read) - k + 1):
            if kIndex.get(read[i:i+k]) == None:
                kIndex[read[i:i+k]] = set([read])  ## set([x]) !
            else:
                kIndex[read[i:i+k]].add(read)
            
    for read in reads:
        for candidate in kIndex[read[-k:]]:
            if candidate != read:
                olen = overlap(read, candidate, min_overlap)
                if olen >= min_overlap:
                    pairs.append([olen, read, candidate])
    pairs.sort(reverse=True)
    return pairs


def greedy_scs_w_index(reads, k, min_overlap):
    pairs = allOverlapPairs(reads, k, min_overlap)
    olen, reada, readb = pairs[0][0], pairs[0][1], pairs[0][2]

    while olen > 0:
        reads.remove(reada)
        reads.remove(readb)
        reads.append(reada + readb[olen:])
        
        pairs = allOverlapPairs(reads, k, min_overlap)
        if len(pairs) == 0:
            print "No more overlapping pairs remain."
            break
        olen, reada, readb = pairs[0][0], pairs[0][1], pairs[0][2]

    return ''.join(reads)
    

### 3. Examples of reads sequences

In [5]:
filename = "ads1_week4_reads.fq"
reads = readFastq(filename)
print "There are total {} read sequences.".format(len(reads))
print "First 6 reads sequences: ", reads[1:6]

There are total 1881 read sequences
First 6 reads sequences:  ['GGAGTACGACTTCAGAGATCTCACTTGGTGTATCAACCCGCCAGAGAGAATCAAATTGGATTATGATCAATACTGTGCAGATGTGGCTGCTGAAGAACTC', 'GCAAATTTTGATCTCTCTTGGCTTCACAATCAATTCAACCATGACCCGAGATGTAGTCATACCCCTCCTCACAAACAACGATCTCTTAATAAGGATGGCA', 'GAGTTAATTGAAGCCCTAGATTACATTTTCATAACTGATGACATACATCTGACAGGGGAGATTTTCTCATTTTTCAGAAGTTTCGGCCACCCCAGACTTG', 'AATGACAGAGACCGCTATGACCATTGATGCTAGGTATGCAGAACTTCTAGGAAGAGTCAGATACATGTGGAAACTGATAGATGGTTTCTTCCCTGCACTC', 'TACGATTTCGACAAGTCGGCATGGGACATCAAAGGGTCGATCGCTCCGATACAACCTACCACCTACAGTGATGGCAGGCTGGTGCCCCAGGTCAGAGTCA']


### 4. Sequence assembly using index-assisted shortest common superstring algorithm

In [2]:
# index assisted overlap matching assembly
# a ~ 10x improvement than without index assisted assembly
# %% time
genome_readsIndexed = greedy_scs_w_index(reads, 30, 1)
len(genome_readsIndexed)

# CPU times: user 3min 24s, sys: 3.74 s, total: 3min 28s
# Wall time: 3min 26s

No more overlapping pairs remain.


15894

### 5. Final assembled complete sequence

In [7]:
print genome_readsIndexed

ACCAAACAAAGTTGGGTAAGGATAGATCAATCAATGATCATATTCTAGTACACTTAGGATTCAAGATCCTATTATCAGGGACAAGAGCAGGATTAGGGATATCCGAGATGGCCACACTTTTGAGGAGCTTAGCATTGTTCAAAAGAAACAAGGACAAACCACCCATTACATCAGGATCCGGTGGAGCCATCAGAGGAATCAAACACATTATTATAGTACCAATTCCTGGAGATTCCTCAATTACCACTCGATCCAGACTACTGGACCGGTTGGTCAGGTTAATTGGAAACCCGGATGTGAGCGGGCCCAAACTAACAGGGGCACTAATAGGTATATTATCCTTATTTGTGGAGTCTCCAGGTCAATTGATTCAGAGGATCACCGATGACCCTGACGTTAGCATCAGGCTGTTAGAGGTTGTTCAGAGTGACCAGTCACAATCTGGCCTTACCTTCGCATCAAGAGGTACCAACATGGAGGATGAGGCGGACCAATACTTTTCACATGATGATCCAAGCAGTAGTGATCAATCCAGGTCCGGATGGTTCGAGAACAAGGAAATCTCAGATATTGAAGTGCAAGACCCTGAGGGATTCAACATGATTCTGGGTACCATTCTAGCCCAGATCTGGGTCTTGCTCGCAAAGGCGGTTACGGCCCCAGACACGGCAGCTGATTCGGAGCTAAGAAGGTGGATAAAGTACACCCAACAAAGAAGGGTAGTTGGTGAATTTAGATTGGAGAGAAAATGGTTGGATGTGGTGAGGAACAGGATTGCCGAGGACCTCTCTTTACGCCGATTCATGGTGGCTCTAATCCTGGATATCAAGAGGACACCCGGGAACAAACCTAGGATTGCTGAAATGATATGTGACATTGATACATATATCGTAGAGGCAGGATTAGCCAGTTTTATCCTGACTATTAAGTTTGGGATAGAAACTATGTATCCTGCTCTTGGACTGCATGAATTTGCTGGTGAGTTATCCACACTTGAGTC

### 6. Conclusion

NCBI Blastn shows that the assembled sequence is that of the complete genome of the RNA Measles virus (GenBank: AB016162.1).